In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np

import xarray as xr

import regionmask

import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import cartopy.feature as cfeat
import matplotlib.patches as mpatches


import glob


import sys
sys.path.append('../')
import utils
import plotting

np.seterr(invalid='ignore'); # disable a warning from matplotlib and cartopy


Bad key "text.kerning_factor" on line 4 in
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


# Concentration Data

In [2]:
ds_MODEL_speciesconc = xr.open_mfdataset('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_final-validation/OutputDir/GEOSChem.SpeciesConc.2016*_0000z.nc4',combine='by_coords')
ds_MODEL_aerosolmass = xr.open_mfdataset('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_final-validation/OutputDir/GEOSChem.AerosolMass.2016*_0000z.nc4',combine='by_coords')
ds_MODEL = xr.merge([ds_MODEL_aerosolmass, ds_MODEL_speciesconc])
ds_MODEL.attrs['name'] = 'normal'

ds_nonuc_speciesconc = xr.open_mfdataset('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_nonuclear/OutputDir/GEOSChem.SpeciesConc.2016*_0000z.nc4',combine='by_coords')
ds_nonuc_aerosolmass = xr.open_mfdataset('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_nonuclear/OutputDir/GEOSChem.AerosolMass.2016*_0000z.nc4',combine='by_coords')
ds_nonuc = xr.merge([ds_nonuc_aerosolmass, ds_nonuc_speciesconc])
ds_nonuc.attrs['name'] = 'nonuc'

datasets = [
    ds_MODEL,
    ds_nonuc
]

In [3]:
species_list = [ #create a list of the species we want to keep
    'NO',
    'NO2',
    'SO2',
    'O3',
    'CH2O',
    'NOx',
    'NH3', 
    'NO3',
    'HNO3',
    'SO4s',
    'SO4'
]
species_tracers = ['SpeciesConc_' + spec for spec in species_list]
species_tracers.append('PM25')

In [4]:
ds = xr.concat([ds_MODEL, ds_nonuc], pd.Index(['normal', 'nonuc'], name='model_name'))
ds['SpeciesConc_NOx'] = (
        ds['SpeciesConc_NO'] + ds['SpeciesConc_NO2'] #sum our NO2 and NO to get NOx
                   )
poll_ds = ds.drop_vars([species for species in ds.data_vars if species not in species_tracers]).rename({'SpeciesConc_' + spec: spec for spec in species_list})

In [5]:
#convert everything to ppbv
for species in species_list:
    poll_ds[f'{species}'] *= 1e9 #convert from mol/mol to ppbv
    poll_ds[f'{species}'].attrs['units'] = 'ppbv'
    
#calculate the NO2/CH2O ratio    
poll_ds[f'CH2O_NO2'] = poll_ds['CH2O']/poll_ds['NO2']
poll_ds[f'CH2O_NO2'].attrs['units'] = 'Ratio CH2O/NO2'

for species in species_list:
    #calculate the differences for species
        poll_ds[f'dif_{species}'] = poll_ds.sel(model_name = 'nonuc')[f'{species}'] - poll_ds.sel(model_name = 'normal')[f'{species}']
        poll_ds[f'dif_{species}'].attrs['units'] = 'ppbv'
        
        #calculate the percent differences for species
        poll_ds[f'percent_dif_{species}'] = (poll_ds[f'dif_{species}']/poll_ds.sel(model_name = 'normal')[f'{species}'])*100
        poll_ds[f'percent_dif_{species}'].attrs['units'] = 'Percent Difference'

#calculate the dif for PM
poll_ds['dif_PM25'] = poll_ds.sel(model_name = 'nonuc')['PM25']- poll_ds.sel(model_name = 'normal')['PM25']
poll_ds[f'dif_PM25'].attrs['units'] = 'PM $\mu$g m$^{-3}$'

#calculate percent dif for PM
poll_ds[f'percent_dif_PM25'] = (poll_ds[f'dif_PM25']/poll_ds.sel(model_name = 'normal')['PM25'])*100
poll_ds[f'percent_dif_PM25'].attrs['units'] = 'Percent Difference'

poll_ds = poll_ds.isel(lev = 0)

poll_ds.attrs['name'] = 'no-nuclear and normal'

In [6]:
poll_ds.to_zarr('nonuc_normal_GC', mode = 'w') #save the dataset 

# HEMCO Data

In [2]:
ds_normal = xr.open_mfdataset('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_final-validation/OutputDir/HEMCO_diagnostics.2016*.nc', combine = 'by_coords')
ds_nonuc = xr.open_mfdataset('../../GCrundirs/type_NA-out_conc-yr_2016-gridmod_nonuclear/OutputDir/HEMCO_diagnostics.2016*.nc', combine = 'by_coords')

ds_normal = ds_normal.isel(lev = 0)
ds_nonuc = ds_nonuc.isel(lev = 0)

ds_emis = xr.concat([ds_nonuc, ds_normal], pd.Index(['nonuc', 'normal'], name='model_name'))

In [3]:
emis_list = ['EmisNO2_Anthro', 'EmisNO_Anthro', 'EmisSO2_Anthro',  'EmisNH3_Total']

In [10]:
emis_ds = ds_emis.drop_vars([species for species in ds_emis.data_vars if species not in emis_list])
emis_ds['EmisNOx_Anthro'] = (
        emis_ds['EmisNO2_Anthro'] + emis_ds['EmisNO_Anthro'] #sum our NO2 and NO to get NOx
                   )
emis_ds['EmisNOx_Anthro'].attrs['units'] = 'kg/m2/s'

In [11]:
emis_ds.to_zarr('emissions_HEMCO', mode = 'w') #save the dataset 